# **Gradio:** Advanced UI

In [ ]:
import gradio as gr
import calendar
from datetime import datetime, timedelta

def agent_feature_calls(month, agent, projectid, features):
    start_time, end_time = get_month_dates(month)
    return agent_lowest_calls(agent, start_time, end_time, projectid, features)

def top_agent_feature_calls(month, agent, projectid, features):
    start_time, end_time = get_month_dates(month)
    return agent_top_calls(agent, start_time, end_time, projectid, features)

def top_diff_features(data):
    data = data.reset_index(drop=True)
    agent_f = eval(data['Agent (Average) Features'][0])
    top_agent_f = eval(data['Top Agents (Average) Features'][0])
    percentage_differences = {key: (abs(top_agent_f.get(key, 0) - agent_f.get(key, 0))/ ((top_agent_f.get(key, 0) + agent_f.get(key, 0))/2))*100  for key in set(top_agent_f)}
    percentage_differences = dict(sorted(percentage_differences.items(), key=lambda x:x[1], reverse=True))
    top_3_features_with_max_difference = list(percentage_differences.keys())[:3]
    return top_3_features_with_max_difference

def get_month_dates(month):
    # Get the current year
    current_year = datetime.now().year
    # Get the first day of the month
    start_date = datetime(current_year, int(month), 1)
    # Get the last day of the month
    _, last_day = calendar.monthrange(current_year, month)
    end_date = datetime(current_year, month, last_day)
    return start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")

def agent_function(month, agent, projectid):
    start_time, end_time = get_month_dates(month)
    agent_data = user_inputs(start_time, end_time, projectid, agent, con)
    report = agent_data["Agent Report"].values[0]
    guidelines = agent_data["Guideline/Tips"].values[0]
    calls = agent_data["Total Calls"].values[0]
    sales = agent_data["Total Sales"].values[0]
    score = agent_data["Score"].values[0]
    state = agent_data["Agent State"].values[0]
    projectId = agent_data["ProjectID"].values[0]
    features = top_diff_features(agent_data)
    top_agents = eval(agent_data["Top Agent Names"].values[0])
    return (
        calls,
        sales,
        score,
        state,
        report,
        guidelines,
        agent_comparison_chart(agent, agent_data),
        agent_talk_chart(agent, agent_data),
        agent_difference_percentages(agent, agent_data),
        sentiment_diff(agent, agent_data),
        top_agents,
        features
    )

with gr.Blocks() as demo:
    gr.Markdown("## Agent Reporting Dashboard")
    # INPUTS
    with gr.Row():
        month_sel = [gr.Dropdown([1,2,3,4,5], label="Select Month")]
        agent_sel = [gr.Dropdown(['Abdul Ali', 'Yono Noyman', 'Aditya Chawla'], label="Select Agent")]
        project_sel = [gr.Text(label="ProjectID", value=168)]
        run_button = gr.Button("Run")
        
    # OUTPUTS    
    with gr.Row():
        calls_output = [gr.Text(label="Total Calls"),
                          gr.Text(label="Total Sales"),
                          gr.Text(label="Score"),
                          gr.Text(label="Agent State")]
    with gr.Row():
        report_output = [gr.Text(label="Agent Performance Report"),
                          gr.Text(label="Training Guidelines/Tips")]
    with gr.Row():
        emotion_graph = [gr.Plot(label="Emotions Graph")]
    with gr.Row():
        talk_graph = [gr.Plot(label="Talk Metrics Graph")]
    with gr.Row():
        differences_graph = [gr.Plot(label="Percentange Differences Graph")]
    with gr.Row():
        sentiment_graph = [gr.Plot(label="Sentiment Differences Graph")]
    with gr.Row():
        top_agents_names = [gr.Text(label="Top 5 Agents")]
        top_feature_diff = [gr.Text(label="Top % Difference Features")]
        top_feature_drop = [gr.Dropdown(top_feature_diff[0].value, label="Select Feature")]
    with gr.Row():
        agent_calls_button = gr.Button("Agent Calls")
        agents_calls = gr.Dataframe(label="Agent's Lowest Calls")
    with gr.Row():
        top_agent_calls_button = gr.Button("Top Agent Calls")
        top_agents_calls = gr.Dataframe(label="Top Agent's Highest Calls")

    # BUTTONS
    inputs = month_sel + agent_sel + project_sel
    outputs = calls_output + report_output + emotion_graph + talk_graph + differences_graph + sentiment_graph + top_agents_names + top_feature_diff
    run_button.click(agent_function, inputs=inputs, outputs=outputs)
    
    agent_calls_inputs = inputs + top_feature_diff
    agent_calls_button.click(agent_feature_calls, inputs=agent_calls_inputs, outputs=agents_calls)  
    
    top_agents_inputs = month_sel + top_agents_names + project_sel + top_feature_diff
    top_agent_calls_button.click(top_agent_feature_calls, inputs=top_agents_inputs, outputs=top_agents_calls) 


demo.launch(share=True)